<a href="https://colab.research.google.com/github/roni762583/bible-data-science.github.io/blob/master/CleanBible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math


In [2]:
#clone data into datafile
! git clone https://github.com/roni762583/bible-data-science.github.io.git
# copy my datafile
! cp /content/bible-data-science.github.io/data/t3utf.dat datafile
# delete unused parts
! rm -rf /content/bible-data-science.github.io/
! ls -la

Cloning into 'bible-data-science.github.io'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 199 (delta 24), reused 0 (delta 0), pack-reused 152
Receiving objects: 100% (199/199), 7.92 MiB | 8.47 MiB/s, done.
Resolving deltas: 100% (101/101), done.
total 5648
drwxr-xr-x 1 root root    4096 Jan 31 16:23 .
drwxr-xr-x 1 root root    4096 Jan 31 16:22 ..
drwxr-xr-x 1 root root    4096 Jan 28 17:15 .config
-rw-r--r-- 1 root root 5763655 Jan 31 16:23 datafile
drwxr-xr-x 1 root root    4096 Jan 20 17:27 sample_data


In [6]:
# get unicode codes for each char in file preserving order and lines stucture 01/25/2021
f = 'א'
ef = f.encode("unicode_escape")
df = ef.decode("unicode_escape")
print(f,ef,df)
print('-----')
file = open('datafile','r')
lineCount = 0
NonHebrewCharsLst = list() # list to hold non-Hebrew block chars
badLines = list() # list for line numbers with non-Hebrew chars to manually review
exceptions = [' ']
nonHebrewLtrCt = 0 # loop counter
for line in file: 
    lineCount += 1
    index = line.rfind('|')+1
    verse = line[:index] # from begining to last occurance of | symbol
    lineText = line[index:].strip().replace('׃','') #strip removes newline char, replace removes sof-pasuk colon 
    
    # range of character code points in unicode Hebrew Block: 0590..05FF; Hebrew
    for char in lineText:
      hexS = hex(ord(char)) # string type, holding hex code of integer representing the Unicode character.
      intS = str(int(hexS,16)).encode("unicode_escape").decode("unicode_escape") # string type, int representing  the Unicode character
      # print('block start: ', int('0x590',16), 'char: ', int(hexS,16), 'block end: ', int('0x5FF',16))
      #break
      bol1 = int(hexS,16)>=int('0x590',16)
      bol2 = int(hexS,16)<=int('0x5FF',16)
      bol3 = bol1 and bol2
      # print(bol3)
      if (bol3): #i.e. if in Hebrew char block
        continue
        print(chr(int(hexS,16))) # hex code for char in
      else: # if char not in Hebrew block of unicode
        if (char in exceptions):
          continue
        nonHebrewLtrCt += 1
        NonHebrewCharsLst.append(intS) # str(int(hexS,16)))
        badLines.append(lineCount)
        # print('on line ', lineCount, 'look for ', )
    
# output results
print('line count: ', lineCount)
print('verse: ', verse)
print('lineText: ', lineText)
encodedVerse = lineText.encode("unicode_escape")
decodedVerse = encodedVerse.decode("unicode_escape")
print('code points: ', encodedVerse)
print('decoded verse: ', decodedVerse)
uniqueChars = list(set(lineText)) # get unique chars list
print('unique chars: ', uniqueChars)
print('') # output separator newline
uniqueNonHebrewCharsLst = list(set(NonHebrewCharsLst))
# print out total count of non-Hebrew block chars, and unique set of charsof this count
print('nonHebrewLtrCt: ', nonHebrewLtrCt, '\nunique list: ', uniqueNonHebrewCharsLst)
# dictionary for line numbers of non-Hebrew occurances
dic = dict()
for c in uniqueNonHebrewCharsLst:
  #print(chr(int(c)))
  if chr(int(c)) in dic:
    dic[chr(int(c))] = dic[chr(int(c))] + 1
  else:
    dic[chr(int(c))] = 1
# print dictionary
#print('dictionary of non-Heb char ct: ', dic)
#print(int('0x5FF',16))
#print(int(hexS,16))
#print()


א b'\\u05d0' א
-----
line count:  23213
verse:  ch2|36|23|
lineText:  כֹּה־אָמַ֞ר כֹּ֣ורֶשׁ׀ מֶ֣לֶךְ פָּרַ֗ס כָּל־מַמְלְכֹ֤ות הָאָ֙רֶץ֙ נָ֣תַן לִ֗י יְהוָה֙ אֱלֹהֵ֣י הַשָּׁמַ֔יִם וְהֽוּא־פָקַ֤ד עָלַי֙ לִבְנֹֽות־לֹ֣ו בַ֔יִת בִּירוּשָׁלִַ֖ם אֲשֶׁ֣ר בִּֽיהוּדָ֑ה מִֽי־בָכֶ֣ם מִכָּל־עַמֹּ֗ו יְהוָ֧ה אֱלֹהָ֛יו עִמֹּ֖ו וְיָֽעַל
code points:  b'\\u05db\\u05bc\\u05b9\\u05d4\\u05be\\u05d0\\u05b8\\u05de\\u05b7\\u059e\\u05e8 \\u05db\\u05bc\\u05b9\\u05a3\\u05d5\\u05e8\\u05b6\\u05e9\\u05c1\\u05c0 \\u05de\\u05b6\\u05a3\\u05dc\\u05b6\\u05da\\u05b0 \\u05e4\\u05bc\\u05b8\\u05e8\\u05b7\\u0597\\u05e1 \\u05db\\u05bc\\u05b8\\u05dc\\u05be\\u05de\\u05b7\\u05de\\u05b0\\u05dc\\u05b0\\u05db\\u05b9\\u05a4\\u05d5\\u05ea \\u05d4\\u05b8\\u05d0\\u05b8\\u0599\\u05e8\\u05b6\\u05e5\\u0599 \\u05e0\\u05b8\\u05a3\\u05ea\\u05b7\\u05df \\u05dc\\u05b4\\u0597\\u05d9 \\u05d9\\u05b0\\u05d4\\u05d5\\u05b8\\u05d4\\u0599 \\u05d0\\u05b1\\u05dc\\u05b9\\u05d4\\u05b5\\u05a3\\u05d9 \\u05d4\\u05b7\\u05e9\\u05c1\\u05bc\\u05b8\\u05de\\u05b7\\

In [ ]:
print(len(uniqueNonHebrewCharsLst))


NameError: ignored

In [ ]:
# Build Tanach string without non-Hebrew letters characters
# Opening file 
file1 = open('datafile', 'r') 
count = 0
s = '' # basic Hebrew chars string
s2 = '' # non-Hebrew chars string
# dataframe for reshaped data
# df = pd.dataframe()
# loop over file line by line, one verse per line 
for line in file1: 
    count += 1
    #print("Line{}: {}".format(count, line.strip())) 
    #print(line[0]) # first character in line
    #print(line.rfind('|')) # last index of pipe '|' character
    line = (line[line.rfind('|')+1:]).replace('׃','') #remove verse and sof-pasuk colon
    line = line.strip() # remove newline char
    line = line.replace(" ", "") # remove spaces
    for c in line:
      if c in {'א', 'ב', 'ג' ,'ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם','נ','ן','ס','ע','פ','ף','צ','ץ','ק','ר','ש','ת'}:
        s += c # concat Hebrew letter to s
      else:
        s2 += c
print (list(set(s))) # get unique chars list
print (s[-1]) # print last letter ל
# Closing files 
file1.close()

['ע', 'ח', 'ף', 'ך', 'מ', 'ן', 'ש', 'ט', 'ה', 'ז', 'נ', 'ץ', 'פ', 'ב', 'י', 'ס', 'ת', 'ד', 'ק', 'ם', 'ר', 'כ', 'ג', 'צ', 'ו', 'ל', 'א']
ל


In [ ]:
# write no spaces string to file - only run once
text_file = open("NoSpacesTanach.dat", "w")
text_file.write(s)
text_file.close()

# delete intermediary datafile
! rm datafile

#see
! ls -la

total 2368
drwxr-xr-x 1 root root    4096 Jan 17 05:33 .
drwxr-xr-x 1 root root    4096 Jan 17 02:30 ..
drwxr-xr-x 1 root root    4096 Jan  8 17:11 .config
-rw-r--r-- 1 root root 2405776 Jan 17 05:33 NoSpacesTanach.dat


In [ ]:
# download result file - only run once
from google.colab import files
files.download('NoSpacesTanach.dat')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# get user input for search term
term1 = input ("Enter Primary Search Term:")
print(term1)

Enter Primary Search Term:ולשתותאתמימי
ולשתותאתמימי


In [ ]:
# find indeces of search term letter in bible text
# from: https://stackoverflow.com/questions/1883980/find-the-nth-occurrence-of-substring-in-a-string
def find_nth_overlapping(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+1)
        n -= 1
    return start

In [ ]:
# ex. 1st occurance of עדנה in string
nthOccurance = 1
index = find_nth_overlapping(s, term1, nthOccurance)
print(s[index-10:index+10])
print("index "+str(index)+" letter "+s[index])


index -1 letter ל


In [ ]:
# try to get some ELS 
# from: https://stackoverflow.com/questions/46503865/getting-every-nth-character-in-the-string-in-python
def GetNthLetters(text, n):
    builtstring = ""
    for i in range(0, len(text)):
        if (i + 1) % n == 0:
            # print(text[i])
            builtstring = builtstring + text[i]
    return builtstring


#text = '1234567890123456789012345678901234567890'

nthLetters = GetNthLetters(s, 2) # 1 or larger. is ELS+1 i.e. no skip is ELS of zero, but here it is one.
print(nthLetters[0:100])

רשתראהםתשיותאץהרהתתובוחךלנתווואהםרפעפימםימאהםהארייווראהםתאריוובלליבןארבןחךיראהםארווחךרללוהעבייקיםחוא


In [ ]:
# Get nth char in string from index j - els string starting at index of open text
def GetELSat(text, n, j):
    text = text[j:]
    print("castrated text: " + text[:50])
    elsstring = ""
    for i in range(0, len(text)):
        if (i + 1) % n == 0:
            # print(text[i])
            elsstring = elsstring + text[i]
    return elsstring

In [ ]:
# func returns indeces of character in string
#from: https://stackoverflow.com/questions/11122291/how-to-find-char-in-string-and-get-all-the-indexes/11122355#11122355
def findIndeces(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

elsMax = 5000

for i in range(0, len(term1)):    # loop over main search term
  for j in range(0, len(s)):      # loop over text
    if(term1[i]==s[j]):           # letter matches
      startIndex = j              # note word start index
      nextTermLetter = term1[i+1] # note next letter in search term
      nextLetterIndeces = findIndeces(s, nextTermLetter) # indeces of 2nd/next letter in search term that appear in text
      
      for k in range(0, len(nextLetterIndeces)): # loop over next letter indeces to find skip
        if (nextLetterIndeces[k]>startIndex && nextLetterIndeces[k]< elsMax): # positive ELS less than max value
          skip = nextLetterIndeces[k] - startIndex
          if(s[k+skip]==term1[i+2])


In [ ]:
# get user main search term
# get 1st letter of search term
# locate 1st letter in text
# count skip to next letter in search term
# if all search term letters are found using same skip, note start index, skip
# continue to next occurance of first letter of search term in text to repeat

# get user input for search term
# term1 = input ("Enter Primary Search Term:")
firstLetter = term1[0]
secondLetter = term1[1]
thirdLetter = term1[2]
# get all indeces of firstLetter in text
# from: https://stackoverflow.com/questions/11122291/how-to-find-char-in-string-and-get-all-the-indexes/11122355#11122355
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]
indecesFirstLtr = find(s, firstLetter)
indecesSecondLtr = find(s, secondLetter)
indecesThirdLtr = find(s, thirdLetter)
print ("1st ltr indx: "+str(indecesFirstLtr))
print ("2nd ltr indx: "+str(indecesSecondLtr))
print ("3rd ltr indx: "+str(indecesThirdLtr))

# check if diff between 1st and 2nd letter index appears in set of diffs btwn 2nd and 3rd, otherwise continue
#for x in indecesFirstLtr:
#  for y in indecesSecondLtr:
#    skip1st2nd = (y - x)
#    expected3rd = y + skip1st2nd
#    if expected3rd in indecesThirdLtr:
 #     print (skip1st2nd)
      # check if next and all letters follow this ELS, yes-note in results, no-continue


# get list of indeces for all subsequent occurances of secondLetter
# get list of differences 


1st ltr indx: [48, 71, 181, 213, 243, 256, 280, 293, 299, 320, 329, 467, 474, 477, 478, 483, 499, 503, 527, 534, 537, 545, 547, 558, 586, 625, 665, 694, 705, 719, 812, 823, 884, 900, 931, 935, 939, 945, 953, 1022, 1090, 1104, 1180, 1258, 1291, 1325, 1443, 1463, 1493, 1498, 1501, 1505, 1522, 1532, 1536, 1539, 1567, 1583, 1607, 1637, 1654, 1710, 1721, 1737, 1751, 1773, 1811, 1844, 1889, 1921, 1935, 1949, 1995, 2040, 2053, 2101, 2122, 2137, 2141, 2148, 2157, 2187, 2339, 2375, 2379, 2402, 2415, 2429, 2433, 2440, 2504, 2509, 2551, 2633, 2662, 2688, 2709, 2744, 2786, 2788, 2792, 2829, 2834, 2882, 2914, 2931, 2976, 3000, 3015, 3051, 3091, 3118, 3138, 3145, 3160, 3178, 3189, 3222, 3237, 3249, 3253, 3266, 3275, 3289, 3353, 3401, 3414, 3442, 3453, 3500, 3516, 3545, 3594, 3627, 3637, 3680, 3688, 3711, 3731, 3743, 3789, 3807, 3843, 3849, 3892, 3901, 3914, 3939, 3948, 3990, 4016, 4058, 4065, 4067, 4086, 4101, 4123, 4127, 4169, 4206, 4220, 4289, 4301, 4377, 4412, 4610, 4631, 4646, 4661, 4710, 4733, 

KeyboardInterrupt: ignored